# MSc Demo Training

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# replace this with the path to your project
%cd /content/drive/MyDrive/Colab Notebooks/DISSERTATION/2_Training

/content/drive/MyDrive/Colab Notebooks/DISSERTATION/2_Training


In [ ]:
# confirm it works by running this cell and checking the output matched the path in the above cell
!pwd

/content/drive/MyDrive/Colab Notebooks/DISSERTATION/2_Training


In [ ]:
# install the stable baselines 3 library
!pip install stable_baselines3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 23.9 MB/s eta 0:00:00


In [ ]:
import gymnasium

# utils imports
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler

# environment imports
import numpy as np
import pandas as pd
import random
import os
import json
import sys
import pickle
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

# training imports
import torch
from stable_baselines3 import PPO  # project 4
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement

# set seed variable
seed = 42

## Utils

In [ ]:
# calculate profit
def calculate_profit(position_size, trade_direction, entry_price, exit_price):

    fixed_t_cost = 0.0001  # hopefully we can increase this at some point
    size_multiplier = 1.0  # this is a hyperparameter we can play around with
    position_size = position_size * size_multiplier

    # fixed transaction cost
    t_cost = fixed_t_cost * position_size

    price_change = (exit_price - entry_price) / entry_price

    if trade_direction == 1:
        profit = price_change * position_size
    elif trade_direction == -1:
        profit = -price_change * position_size
    else:
        return 0, 0

    trade_profit = profit - t_cost
    marginal_return = trade_profit / (position_size * entry_price)

    return trade_profit, marginal_return

# get datasets
def get_data(pair, window, theta):

    train_df = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/DISSERTATION/1_DataTransformation/TransformedData/{theta}/{pair}/Window_{window}/train.parquet.gzip')
    val_df = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/DISSERTATION/1_DataTransformation/TransformedData/{theta}/{pair}/Window_{window}/validation.parquet.gzip')
    test_df = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/DISSERTATION/1_DataTransformation/TransformedData/{theta}/{pair}/Window_{window}/test.parquet.gzip')

    # get ask prices
    train_asks = train_df['Ask'].values
    val_asks = val_df['Ask'].values
    test_asks = test_df['Ask'].values

    # get bid prices
    train_bids = train_df['Bid'].values
    val_bids = val_df['Bid'].values
    test_bids = test_df['Bid'].values

    # train_df, val_df, test_df = normalize_dataframes(train_df, val_df, test_df)
    train_df, val_df, test_df = manual_normalise(train_df, val_df, test_df)

    # temp remove to match old approach
    train_df = train_df.drop(['Direction', 'Ask', 'Bid', 'Spread'], axis=1)
    val_df = val_df.drop(['Direction', 'Ask', 'Bid', 'Spread'], axis=1)
    test_df = test_df.drop(['Direction', 'Ask', 'Bid', 'Spread'], axis=1)

    if 'JPY' in pair:
        train_df[['Start', 'DCC']] = train_df[['Start', 'DCC']] / 100
        val_df[['Start', 'DCC']] = val_df[['Start', 'DCC']] / 100
        test_df[['Start', 'DCC']] = test_df[['Start', 'DCC']] / 100

    # print(train_df.head)
    # print(train_df.columns.to_list())

    train_data = train_df.values
    val_data = val_df.values
    test_data = test_df.values

    data_dict = {
        'train_data': train_data,
        'train_asks': train_asks,
        'train_bids': train_bids,
        'val_data': val_data,
        'val_asks': val_asks,
        'val_bids': val_bids,
        'test_data': test_data,
        'test_asks': test_asks,
        'test_bids': test_bids
    }

    print("------- Data -------")
    print(f"Train Shape: {train_data.shape}")
    print(f"Validation Shape: {val_data.shape}")
    print(f"Test Shape: {test_data.shape}")

    return data_dict

# percentage change
def pct_change(old_value, new_value):
    change = new_value - old_value
    percentage_change = (change / old_value)
    return percentage_change

# shift by n, new start values are replace by np.nan and end values are discarded
def shift(array, shift):
    return np.concatenate(([np.nan] * shift, array[:-shift]))

# rolling window generator, left over events are discarded
def rolling_window(df, window_size, shift):
    for i in range(0, len(df) - window_size + 1, shift):
        yield df.iloc[i:i+window_size]

# take train, validation and test sets and normalise based on training data
def normalize_dataframes(train_df, val_df, test_df):
    # create a MinMaxScaler object
    scaler = MinMaxScaler()

    # fit the scaler on the train DataFrame
    scaler.fit(train_df)

    # normalize each DataFrame using the fitted scaler
    train_normalized = pd.DataFrame(scaler.transform(train_df), columns=train_df.columns)
    val_normalized = pd.DataFrame(scaler.transform(val_df), columns=val_df.columns)
    test_normalized = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns)

    return train_normalized, val_normalized, test_normalized

def manual_normalise(train_df, val_df, test_df):

    DC_start_end = train_df[['Start', 'DCC']]
    train_transformed = (train_df - train_df.min()) / (train_df.max() - train_df.min())
    train_transformed[['Start', 'DCC']] = DC_start_end

    DC_start_end = val_df[['Start', 'DCC']]
    val_transformed = (val_df - train_df.min()) / (train_df.max() - train_df.min())
    val_transformed[['Start', 'DCC']] = DC_start_end

    DC_start_end = test_df[['Start', 'DCC']]
    test_transformed = (test_df - train_df.min()) / (train_df.max() - train_df.min())
    test_transformed[['Start', 'DCC']] = DC_start_end

    return train_transformed, val_transformed, test_transformed


# trend class
class Trend(object):
    def __init__(self, direction, DC_start, DCC, OS_end, DC_start_index, DCC_index, OS_end_index):
        self.direction, self.DC_start, self.DCC, self.OS_end = direction, DC_start, DCC, OS_end
        self.DC_start_index, self.DCC_index, self.OS_end_index = DC_start_index, DCC_index, OS_end_index

        self.data_dict = {
                'Direction': self.direction,
                'Start': round(self.DC_start, 6),
                'DCC': round(self.DCC, 6),
                'End': round(self.OS_end, 6),
                'Start Index': round(self.DC_start_index, 6),
                'DCC Index': round(self.DCC_index, 6),
                'End Index': round(self.OS_end_index, 6),
            }

    def __str__(self):
        return str(self.data_dict)

## Environment

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/DISSERTATION/params.json', 'r') as f:
    params = json.load(f)

class DirectionalChangeEnv(gymnasium.Env):
    def __init__(self, env_config):

        # get data from config
        data = env_config['data']
        asks = env_config['asks']
        bids = env_config['bids']

        self.full_data = data
        self.full_asks = asks
        self.full_bids = bids

        # init state params
        self.context_length = int(params['training']['context_length'])
        self.lag = int(params['training']['lag'])
        try:
            self.start_index = random.randint(0, len(self.full_data) - (self.context_length + 1))
        except:
            self.start_index = 0
        self.end_index = self.start_index + self.context_length

        # init data
        self.data = self.full_data[self.start_index: self.end_index]
        self.asks = self.full_asks[self.start_index: self.end_index]
        self.bids = self.full_bids[self.start_index: self.end_index]

        # init state
        self.i = self.lag - 1
        self.state = self.data[0:self.i + 1]  # (lag, 32)
        self.ask_price = asks[self.i]  # (1)
        self.bid_price = bids[self.i]  # (1)

        # set action and observation space
        self.action_space = gymnasium.spaces.Discrete(2)  # buy, sell
        self.observation_space = gymnasium.spaces.Box(0, 2, shape=self.state.shape)  # DC start, DC end for last 5 timesteps

        # init simulation params
        self.n_prices = len(self.data) - self.lag
        self.balance = 100
        self.entry_price = None
        self.position_size = None
        self.in_position = 0  # -1 for short, 0 for no, 1 for long
        self.trading_log = []

    def step(self, action):

        self.i += 1
        self.n_prices -= 1
        self.state = self.data[self.i - (self.lag-1) : self.i + 1]
        self.ask_price = self.asks[self.i]
        self.bid_price = self.bids[self.i]
        self.mid_price = (self.ask_price + self.bid_price) / 2

        # reward function
        if self.in_position == 0:
            if action == 0:  #  buy
                self.entry_price = self.mid_price  # long at ask price
                self.position_size = self.balance
                self.in_position = 1
                reward = 0
            elif action == 1:  # sell
                self.entry_price = self.mid_price  # short at bid price
                self.position_size = self.balance
                self.in_position = -1
                reward = 0

        elif self.in_position == -1:
            if action == 0:  #  buy
                profit, _ = calculate_profit(self.position_size, self.in_position,
                                          self.entry_price, self.mid_price)  # exit short position at ask price
                self.trading_log.append({'Trade Index': self.i,
                                         'Position Size': self.position_size,
                                         'Trade Type': 'Short',
                                         'Entry Price': self.entry_price,
                                         'Exit Price': self.mid_price,
                                         'Profit': profit})
                reward = profit
                self.balance += profit
                self.in_position = 0
                self.position_size = None
                self.entry_price = None
            elif action == 1:  # sell
                reward = 0

        elif self.in_position == 1:
            if action == 0:  #  buy
                reward = 0
            elif action == 1:  # sell
                profit, _ = calculate_profit(self.position_size, self.in_position,
                                          self.entry_price, self.mid_price)  # exit long position as bid price
                self.trading_log.append({'Trade Index': self.i,
                                         'Position Size': self.position_size,
                                         'Trade Type': 'Long',
                                         'Entry Price': self.entry_price,
                                         'Exit Price': self.mid_price,
                                         'Profit': profit})
                reward = profit
                self.balance += profit
                self.in_position = 0
                self.position_size = None
                self.entry_price = None

        if self.n_prices <= 0 or self.balance <= 0:
            done = True
            if self.in_position != 0:

                if self.in_position == -1:
                    trade_type = 'Short'
                    exit_price = self.mid_price
                elif self.in_position == 1:
                    trade_type = 'Long'
                    exit_price = self.mid_price

                profit, _ = calculate_profit(self.position_size, self.in_position,
                                            self.entry_price, exit_price)
                reward = profit

                self.trading_log.append({'Trade Index': self.i,
                                         'Position Size': self.position_size,
                                         'Trade Type': trade_type,
                                         'Entry Price': self.entry_price,
                                         'Exit Price': exit_price,
                                         'Profit': profit})
                self.balance += profit
                self.in_position = 0
        else:
            done = False

        info = {'balance': self.balance,
                'trading_log': self.trading_log}

        return self.state, reward, done, False, info

    def reset(self, seed=seed, options=None):

        # generate new training set
        try:
            self.start_index = random.randint(0, len(self.full_data) - (self.context_length + 1))
        except:
            self.start_index = 0
        self.end_index = self.start_index + self.context_length
        self.data = self.full_data[self.start_index: self.end_index]
        self.asks = self.full_asks[self.start_index: self.end_index]
        self.bids = self.full_bids[self.start_index: self.end_index]

        # reset episode variables
        self.i = self.lag - 1
        self.state = self.data[0:self.i + 1]
        self.ask_price = self.asks[self.i]
        self.bid_price = self.bids[self.i]
        self.n_prices = len(self.data) - self.lag
        self.balance = 100
        self.entry_price = None
        self.position_size = None
        self.in_position = 0  # -1 for short, 0 for no, 1 for long
        self.trading_log = []

        return self.state, {}



## Training

In [ ]:
from stable_baselines3 import PPO, A2C, DQN, TD3

In [ ]:
class ModelTrainer(object):
    """class to run training of model"""
    def __init__(self, theta, model_name):

        self.theta = theta
        self.model_name = model_name

        model_obj_dict = {"PPO": PPO, "A2C": A2C, "DQN": DQN, "TD3": TD3}

        self.model_obj = model_obj_dict[model_name]

        # get data
        data_dict = get_data(pair, window, theta)
        train_data = data_dict['train_data']
        train_asks = data_dict['train_asks']
        train_bids = data_dict['train_bids']
        val_data = data_dict['val_data']
        val_asks = data_dict['val_asks']
        val_bids = data_dict['val_bids']

        # configure training parameters
        env_config = {
                'data': train_data,
                'asks': train_asks,
                'bids': train_bids
            }

        # configure algorithm
        log_path = f"./Logs/{model_name}/{self.theta}/{pair}/Window_{window}"
        env = DirectionalChangeEnv(env_config)
        self.model = self.model_obj('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

    def save_checkpoint(self):
        # save model
        model_folder = f'./Models/{self.model_name}/{self.theta}/{pair}/'
        self.model.save(model_folder + f'{self.model_name}_Window_{window}')
        print('model saved')

    def train(self):
        self.model.learn(total_timesteps=200000)  # change timesteps to e.g. 10000 for a shorted training process to check it's working


## Run Experiment

In [ ]:
# set seeds - this is to ensure reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# set script variables
pair = str('USDCAD')  # run for all pairs: AUDUSD, EURGBP, EURUSD and USDCAD
window = int(18) # run for all windows (see e.g. /1_DataTransformation/TransformedData/0.00013/AUDUSD/ for list of windows)
theta = float(0.00023)  # run for all thresholds: 0.00013, 0.00017, 0.00023
model_name = 'DQN'  # PPO, A2C, DQN

# create experiment
trainer = ModelTrainer(theta, model_name)

# train the model
trainer.train()

# save model
trainer.save_checkpoint()

------- Data -------
Train Shape: (2097, 30)
Validation Shape: (1064, 30)
Test Shape: (531, 30)
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./Logs/DQN/0.00023/USDCAD/Window_18/DQN_3
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 995      |
|    ep_rew_mean      | -3.37    |
|    exploration_rate | 0.811    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 370      |
|    time_elapsed     | 10       |
|    total_timesteps  | 3980     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 995      |
|    ep_rew_mean      | -3.33    |
|    exploration_rate | 0.622    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 413      |
|    time_elapsed     | 19       |
|    total_timesteps  | 7960     |
----------------------